Introduction
============

This notebook facilitates using the RiverScape Python package (Straatsma
and Kleinhans, 2018) to parameterize and position landscaping measures
and update the input data for the two-dimensional (2D) flow model
Delft3D Flexible Mesh (DFM).

For the current notebook version, we would use the River Waal, which is
the main distributary of the River Rhine in the Netherlands.

Requirement (Python modules/packages)
-------------------------------------

To run this notebook, please import the following Python modules.

In [ ]:
%load_ext autoreload
%autoreload 2

# import standard modules
import os
import sys
import string
import subprocess
import time
import math
import pprint as pp

# import required modules/packages (which may require installation)
import numpy as np
import pandas as pd
import geopandas as gpd
from scipy.spatial import Delaunay
from shapely.geometry import MultiLineString
from shapely.ops import cascaded_union, polygonize


from collections import OrderedDict
import pcraster as pcr

Please also make sure that this notebook file is in the same folder as
the RiverScape Python module files that must be loaded. You can then
import the RiverScape Python module:

In [ ]:
from riverscape import *
import geoviews
geoviews.extension('bokeh')

Input and output folders
========================

In the following please define the input and output folders.

In [ ]:
# Default locations
input_dir    = '/home/schmi109/development/projects/RiverScape/input_files/input' #os.getcwd()
output_dir   = os.getcwd()
scratch_dir  = os.path.join(output_dir, "tmp")


You may also want to set the folder, interactively

In [ ]:
# input_dir  = select_directory()

In [ ]:
# output_dir = select_directory()

In [ ]:
# make scratch directory and go to this folder
pcrRecipes.make_dir(scratch_dir)
os.chdir(scratch_dir)

# print some statements about the folder locations:
msg = "The input folder is set on   : {}".format(input_dir)
print(msg)
msg = "The output folder is set on  : {}".format(output_dir)
print(msg)
msg = "The scratch folder is set on : {}".format(scratch_dir)
print(msg)

Start processing/calculation
============================

To start processing, please load the following cells in order to set and
test some basic configuration.

In [ ]:
# go to the scratch folder
pcrRecipes.make_dir(scratch_dir)
os.chdir(scratch_dir)

# set global option for pcraster such that length of cells is computed in true length of cells
pcr.setglobaloption('unittrue')

# set the pcraster clone map
current_dir = os.path.join(input_dir, 'reference_maps')
pcr.setclone(os.path.join(current_dir, 'clone.map'))

Loading input files
-------------------

By running the following cells, the input files would be read. These
input files consist of the following attributes of current/existing
condition.

-   main\_dike: current/existing river embankment properties,
    e.g. length, volume and height
-   minemb: minor embankment properties, e.g. length, volume and height
-   groynse: groyne properties, e.g. length, volume and height
-   hydro: hydrodynamics (delft3d-fm) attributes, e.g. chezy, nikuradse,
    specific discharge, velocity, water depth, water level, etc.
-   mesh: delft3d mesh
-   axis: river attributes, e.g. location, radius, turning direction,
    velocity, water depth, water level, etc.
-   geom: river geometry attributes, e.g. clone, dem,
    dist\_to\_main\_dike, dist\_to\_groyne\_field,
    dist\_to\_main\_channel, flpl\_width, flpl\_narrow, flpl\_wide,
    main\_channel\_width, river\_side, shore\_line
-   lulc: land use and land cover attributes, e.g. backwaters, ecotopos,
    floodplain, groyne\_field, main\_channel, trachytopes, sections,
    winter\_bed, real\_estate\_value

For further information about them, please check Menno and Kleinhans
(2018), see e.g. their Table 2.

In [ ]:
# change to the 'current_dir' (input data) for reading/importing input data
os.chdir(current_dir)


#TODO: Check why do we need this? It seems that os.path.join does not work?

# reading current/existing river embankment properties for main dikes, minor embankments and groynes
# - for each, this will return location, length, volume, and height
main_dike = msr.read_dike_maps(current_dir, 'main_dike')
minemb    = msr.read_dike_maps(current_dir, 'minemb')
groynes   = msr.read_dike_maps(current_dir, 'groyne')

# reading Hydrodynamics (delft3d-fm) attributes
# - chezy, nikuradse, specific discharge, velocity, water depth, water level, etc
hydro = msr.read_hydro_maps(current_dir)
# - hydrodynamic mesh
mesh = msr.read_mesh_maps(current_dir)

# reading RiverScape attributes
# - axis: location, radius, turning_direction, rkm, rkm_point, rkm_line, rkm_full
axis = msr.read_axis_maps(current_dir)
# - geometry: clone, dem, distance to main dike, distance to groyne, distance to main channel, floodplain widths, contiguous narrow floodplain, contiguous wide floodplain
geom = msr.read_geom_maps(current_dir)
# - land use land cover attributes
lulc = msr.read_lulc_maps(current_dir)

msg = "Files should be already read from: " + current_dir
print("\n")
print(msg)

### Listing attributes/variables

After succesfully reading the files/maps, you should be able to list the
attributes/variables of main\_dike, minemb, groynes, hydro, mesh, axis,
geom, and lulc in the following.

In [ ]:
for obj in ["main_dike", "minemb", "groynes", "hydro", "mesh", "axis", "geom", "lulc"]:
    print("\n")
    print(obj)
    print(vars()[obj].__dict__.keys())

### Visualizing existing attributes

You may want to check existing attributes by plotting the corresponding
raster maps. You can use the plot function for that, for example to plot
the digital elevation model type:

``` python
plot(geom.dem)
```

In [ ]:
# plot river main channel width
plot(geom.main_channel_width)

In [ ]:
# plot height of main_dike
plot(main_dike.height)

In [ ]:
# plot flpl_wide IDs
plot(geom.flpl_wide)

<br>

Initiating the River and Its Measures
-------------------------------------

Given the aforementioned attributes, the River Waal and its current
measured would be initiated by executing the following cells.

In [ ]:
# The River Wall is initiated based on the aforementioned given attributes.
waal = msr.River('Waal', axis, main_dike, minemb, groynes, hydro,
                         mesh, lulc, geom)

In [ ]:
# Initiate the Measures for the River.
waal_msr = msr.RiverMeasures(waal)

Configuration for the Measures
==============================

First, you can have a look at the current specification of the ecotope
and trachytope classes present in the area. The floodplain sections are
depicted as well. Ecotopes are … Trachytopes are …

Open the maps with the following command.

In [ ]:
plot_eco_trachy_sec()

Side channel measure:
---------------------

### Make your own side channel properties:

<!-- The following properties are the default/setting configuration to the Measures. -->

The measures are configured with certain properties. You can inspect and
change a few of them:

In [ ]:
settings = measures_settings()

You may want to modify the side channel properties using the following
interactive cell. Note that height and width are not true to scale.

In [ ]:
channel_values = channel_properties()

Load the following cell to use your configuration.

In [ ]:
settings['channel_width'] = channel_values.kwargs['width']
settings['channel_depth'] = channel_values.kwargs['depth']
settings['channel_slope'] = channel_values.kwargs['slope']
pp.pprint(settings)
waal_msr.settings = settings

### Selecting the mask/region:

Please select the areas where you want to perform this measure.

Please also give a label for this measure.

In [ ]:
label = 'custom_label'

In [ ]:
# Default mask and label
#label = 'everywhere'
#mask = pcr.boolean(1)

# TODO: Make interactive input for selecting mask for every measure.

# - some examples to select a limited mask region

#label = "large_sections"
#mask = pcr.ifthen(pcr.areaarea(waal.geom.flpl_wide) > 1e6, pcr.boolean(1.0))

# -- this will return floodplain with IDs < 5
#label = "edwin_sections"
#mask = pcr.ifthen(pcr.scalar(waal.geom.flpl_wide) < 5, pcr.boolean(1.0))

# - plot the chosen mask
#plot(mask)
#chosen_flpl_wide = pcr.ifthen(mask, waal.geom.flpl_wide)
#plot(chosen_flpl_wide)

In [ ]:

selection = select_area(waal.geom.flpl_wide)


In [ ]:
mask = generate_mask(waal.geom.flpl_wide, selection)


In [ ]:
plot(mask)

In [ ]:
chosen_flpl_wide = pcr.ifthen(mask, waal.geom.flpl_wide)
plot(chosen_flpl_wide)

### Implementing the measure

By running the following cell, the measure will be implemented.

In [ ]:
# measure by side channel constrcution
# - this includes looping over flood plain IDs
chan_msr = waal_msr.side_channel_measure(settings, mask = mask, ID = label)

### Exploring the measure

You can explore the measure implemented by running the following
plotting cell. Plotting may take a while before completed.

In [ ]:
# plot/explore side channel measure
chan_msr.plot()

Flood plain lowering measure:
-----------------------------

Please set the ID/label for this measure and set the mask where you want
to introduce this measure.

In [ ]:
mask = pcr.boolean(1)
ID = 'everywhere'

In [ ]:
# floodplain lowering measure
lowering_msr = waal_msr.lowering_measure(settings, mask=mask, ID=ID)

In [ ]:
# plot/explore floodplain lowering measure
lowering_msr.plot()

Groyne lowering measure:
------------------------

Please set the ID/label for this measure and set the mask where you want
to introduce this measure.

In [ ]:
mask = pcr.boolean(1)
ID = 'everywhere'

In [ ]:
# groyne lowering measure
groyne_low_msr = waal_msr.groyne_lowering_msr(settings, mask=mask, ID=ID)

In [ ]:
# plot/explore groyne lowering measure
groyne_low_msr.plot()

Minor embankment lowering measure:
----------------------------------

Please set the ID/label for this measure and set the mask where you want
to introduce this measure.

In [ ]:
mask = pcr.boolean(1)
ID = 'everywhere'

In [ ]:
# measure by minor embankment lowering
minemb_low_msr = waal_msr.minemb_lowering_msr(settings, mask=mask, ID=ID)

In [ ]:
# plot/explore minor embankment lowering
minemb_low_msr.plot()

Main dike raising measure:
--------------------------

Please set the ID/label for this measure and set the mask where you want
to introduce this measure.

In [ ]:
mask = pcr.boolean(1)
ID = 'everywhere'

In [ ]:
# measure by main dike raising
main_dike_raise_msr = waal_msr.main_dike_raising_msr(settings, mask=mask, ID=ID)

In [ ]:
# plot/explore main dike raising measure
main_dike_raise_msr.plot()

Roughness smoothing measure:
----------------------------

Please set the ID/label for this measure and set the mask where you want
to introduce this measure.

In [ ]:
mask = pcr.boolean(1)
ID = 'everywhere'

In [ ]:
# measure by roughness smoothing
smooth_msr = waal_msr.smoothing_measure(settings, mask=mask, ID=ID)

In [ ]:
# plot/explore measure by roughness smoothing
smooth_msr.plot()

Saving measures to the disk
===========================

In [ ]:
# list of measures
msr_list = [groyne_low_msr, minemb_low_msr,
            main_dike_raise_msr, lowering_msr, chan_msr, smooth_msr]
# - preparing the directory
msr_root_dir = os.path.join(output_dir, 'measures_ensemble/maps')
pcrRecipes.make_dir(msr_root_dir)
for measure in msr_list:
    msr.write_measure(measure, msr_root_dir)